In [ ]:
!pip install opencv-python cvlib pandas matplotlib

import cv2
import cvlib as cv
from cvlib.object_detection import draw_bbox
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import os

os.makedirs("capturas", exist_ok=True)
os.makedirs("logs", exist_ok=True)
os.makedirs("images", exist_ok=True)

log_path = "logs/eventos.csv"
if not os.path.exists(log_path):
    with open(log_path, "w") as f:
        f.write("timestamp,evento,clase,confianza\n")

conteo_objetos = {}

cap = cv2.VideoCapture(0)
frame_count = 0
captura_guardada = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    bbox, label, conf = cv.detect_common_objects(frame, confidence=0.4, model='yolov4-tiny')
    output_image = draw_bbox(frame, bbox, label, conf)

    conteo_frame = {}
    for l in label:
        conteo_frame[l] = conteo_frame.get(l, 0) + 1
        conteo_objetos[l] = conteo_objetos.get(l, 0) + 1

    if 'person' in label:
        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        conf_person = conf[label.index('person')]
        captura_guardada += 1
        filename = f"capturas/persona_{captura_guardada}.jpg"
        cv2.imwrite(filename, frame)


        with open(log_path, "a") as f:
            f.write(f"{timestamp},Persona detectada,person,{conf_person:.2f}\n")
            f.write(f"{timestamp},Captura guardada,person,{conf_person:.2f}\n")


    cv2.imshow("Monitoreo", output_image)


    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


plt.figure(figsize=(10, 5))
plt.bar(conteo_objetos.keys(), conteo_objetos.values(), color='skyblue')
plt.title("Conteo total de objetos detectados")
plt.xlabel("Objeto")
plt.ylabel("Cantidad")
plt.tight_layout()
plt.savefig("images/1.jpg")


df = pd.read_csv(log_path)
person_logs = df[df['clase'] == 'person']
times = pd.to_datetime(person_logs['timestamp'])

plt.figure(figsize=(10, 4))
plt.plot(times, range(len(times)), marker='o', linestyle='-')
plt.title("Detecciones de personas en el tiempo")
plt.xlabel("Tiempo")
plt.ylabel("Evento #")
plt.grid(True)
plt.tight_layout()
plt.savefig("images/2.jpg")
